In [1]:
import os
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import wave
import csv
import tensorflow as tf
import tensorflow_hub as hub
import librosa
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
import tensorflow_io as tfio
import scipy

C:\ProgramData\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
_ = tf.keras.utils.get_file('car.zip',
                        'https://github.com/Boden448/Sound/archive/main.zip',

                        cache_dir='./',
                        cache_subdir='masses',
                        extract=True)

In [3]:

csv_dir = "./masses/Sound-main/meta/"
audio_dir = './masses/Sound-main/DOA_audio/'

# Read the CSV file into a Pandas DataFrame
csv_file = os.path.join(csv_dir, 'DOA.csv')
df = pd.read_csv(csv_file)

In [4]:
audio_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith('.wav')]
audio_data = []
for f in audio_files:
    audio, sr = librosa.load(f, sr=44100, mono=True)
    audio_data.append(audio)

max_length = 44100
padded_audio_data = []
for audio in audio_data:
    if len(audio) >= max_length:
        padded_audio_data.append(audio[:max_length])
    else:
        padded_audio_data.append(np.pad(audio, (0, max_length - len(audio)), mode='constant'))

padded_audio_data = np.array(padded_audio_data)
csv_data = df[['x', 'z', 'theta']].values
pos_data = df[['x', 'z']].values
theta_data = df['theta'].values


In [5]:
input_audio = tf.keras.layers.Input(shape=(max_length, 1))
lstm1 = tf.keras.layers.LSTM(128)(input_audio)
dense1 = tf.keras.layers.Dense(64, activation='softmax')(lstm1)
input_csv = tf.keras.layers.Input(shape=(3,))
concat = tf.keras.layers.Concatenate()([dense1, input_csv])
dense2 = tf.keras.layers.Dense(32, activation='softmax')(concat)
output_pos = tf.keras.layers.Dense(2, activation='linear')(dense2)
output_theta = tf.keras.layers.Dense(1, activation='linear')(dense2)



In [6]:
model = tf.keras.models.Model(inputs=[input_audio, input_csv], outputs=[output_pos, output_theta])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [7]:
model.fit(x=[padded_audio_data.reshape(-1, max_length, 1), csv_data],
          y=[pos_data, theta_data],
          epochs=35,
          batch_size=8,
          steps_per_epoch=len(audio_files)//8,
          validation_split=0.2)

Epoch 1/35
3/3 [==============================] - 148s 52s/step - loss: 430.1891 - dense_2_loss: 430.1891 - dense_3_loss: 6.4938e-06 - dense_2_accuracy: 0.3158 - dense_3_accuracy: 0.0526 - val_loss: -248.2779 - val_dense_2_loss: -248.2779 - val_dense_3_loss: 1.9073e-05 - val_dense_2_accuracy: 0.0000e+00 - val_dense_3_accuracy: 0.0000e+00
Epoch 2/35
3/3 [==============================] - 172s 61s/step - loss: 392.1061 - dense_2_loss: 392.1061 - dense_3_loss: 6.4938e-06 - dense_2_accuracy: 0.6316 - dense_3_accuracy: 0.0526 - val_loss: -257.8406 - val_dense_2_loss: -257.8407 - val_dense_3_loss: 1.9073e-05 - val_dense_2_accuracy: 0.0000e+00 - val_dense_3_accuracy: 0.0000e+00
Epoch 3/35
3/3 [==============================] - 123s 35s/step - loss: 372.8672 - dense_2_loss: 372.8672 - dense_3_loss: 6.4938e-06 - dense_2_accuracy: 0.6842 - dense_3_accuracy: 0.0526 - val_loss: -267.4777 - val_dense_2_loss: -267.4778 - val_dense_3_loss: 1.9073e-05 - val_dense_2_accuracy: 0.0000e+00 - val_dense_3_a

Epoch 25/35
3/3 [==============================] - 194s 59s/step - loss: -413.8620 - dense_2_loss: -413.8620 - dense_3_loss: 6.4938e-06 - dense_2_accuracy: 0.5263 - dense_3_accuracy: 0.0526 - val_loss: -5777.3364 - val_dense_2_loss: -5777.3364 - val_dense_3_loss: 1.9073e-05 - val_dense_2_accuracy: 0.2000 - val_dense_3_accuracy: 0.0000e+00
Epoch 26/35
3/3 [==============================] - 199s 60s/step - loss: -414.2225 - dense_2_loss: -414.2225 - dense_3_loss: 6.4938e-06 - dense_2_accuracy: 0.5263 - dense_3_accuracy: 0.0526 - val_loss: -5777.1758 - val_dense_2_loss: -5777.1758 - val_dense_3_loss: 1.9073e-05 - val_dense_2_accuracy: 0.2000 - val_dense_3_accuracy: 0.0000e+00
Epoch 27/35
3/3 [==============================] - 201s 61s/step - loss: -414.6464 - dense_2_loss: -414.6464 - dense_3_loss: 6.4938e-06 - dense_2_accuracy: 0.5263 - dense_3_accuracy: 0.0526 - val_loss: -5777.0845 - val_dense_2_loss: -5777.0845 - val_dense_3_loss: 1.9073e-05 - val_dense_2_accuracy: 0.2000 - val_dense_

In [8]:
test_input_audio = np.random.rand(1, 44100, 1)
test_input_csv = np.array([[500, 0, 0]])
test_output_pos, test_output_theta = model.predict([test_input_audio, test_input_csv])
print('Predicted position:', test_output_pos)
print('Predicted theta:', test_output_theta)


1/1 [==============================] - 2s 2s/step
Predicted position: [[0.46735442 0.14807877]]
Predicted theta: [[-0.1696058]]
